In [9]:
#q1
##q1
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

df = pd.read_csv(r"/content/RFMiD_Training_Labels.csv")
X = df.drop(columns=[c for c in ["ID", "Disease_Risk"] if c in df.columns])
y = df["Disease_Risk"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
base_models = [("rf", RandomForestClassifier(n_estimators=100, random_state=42)),
               ("gb", GradientBoostingClassifier(random_state=42)),
               ("svc", SVC(probability=True, random_state=42))]
meta_model = LogisticRegression()
stacking = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)
stacking.fit(X_train, y_train)
y_pred = stacking.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        80
           1       1.00      1.00      1.00       304

    accuracy                           1.00       384
   macro avg       1.00      1.00      1.00       384
weighted avg       1.00      1.00      1.00       384

[[ 80   0]
 [  0 304]]


In [6]:
#q2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

df = pd.read_csv(r"/content/RFMiD_Training_Labels.csv")
X = df.drop(columns=[c for c in ["ID", "Disease_Risk"] if c in df.columns])
y = df["Disease_Risk"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
base_models = [("rf", RandomForestClassifier(n_estimators=100, random_state=42)),
               ("gb", GradientBoostingClassifier(random_state=42)),
               ("svc", SVC(probability=True, random_state=42))]
meta_model = LogisticRegression()
stacking = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)
pipeline = Pipeline([("scaler", StandardScaler()), ("stack", stacking)])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 1.0


In [7]:
#q3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from lime.lime_tabular import LimeTabularExplainer

df = pd.read_csv(r"/content/RFMiD_Training_Labels.csv")
X = df.drop(columns=[c for c in ["ID", "Disease_Risk"] if c in df.columns])
y = df["Disease_Risk"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
base_models = [("rf", RandomForestClassifier(n_estimators=100, random_state=42)),
               ("gb", GradientBoostingClassifier(random_state=42)),
               ("svc", SVC(probability=True, random_state=42))]
meta_model = LogisticRegression()
stacking = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)
pipeline = Pipeline([("scaler", StandardScaler()), ("stack", stacking)])
pipeline.fit(X_train, y_train)
explainer = LimeTabularExplainer(training_data=X_train.values,
                                 feature_names=X.columns.tolist(),
                                 class_names=[str(c) for c in pipeline.classes_],
                                 mode="classification")
sample = X_test.iloc[0].values
exp = explainer.explain_instance(sample, pipeline.predict_proba, num_features=10)
print(exp.as_list())


[('DR <= 0.00', -0.4121296794615103), ('MH <= 0.00', -0.3961986415050334), ('ERM <= 0.00', -0.36461508703830997), ('CF <= 0.00', -0.34705904947821964), ('RT <= 0.00', -0.32906321136578215), ('PRH <= 0.00', -0.21591319200452982), ('BRAO <= 0.00', -0.215510819431322), ('TD <= 0.00', -0.17567253481267475), ('HPED <= 0.00', -0.17045765426719678), ('CWS <= 0.00', -0.11158952357316708)]


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
